In [7]:
import pandas as pd

from networkie.gen.Custom import LoadFromFile
c = LoadFromFile()
g = c.from_in_class_network('dataset/In-class_network.txt')

from networkie.utils.Measures import Node
d = Node()
d.degree_dist(g)

#3.(b-1)---------------------------------
print('nodes :',g.order())

#3.(b-2)---------------------------------
print('edges :',g.size())

#3.(b-3)---------------------------------
print('average degree :',g.size()*2/g.order())

#3.(b-4)---------------------------------
path = 0
n = 0
for i in range(g.order()):
    a = list(range(g.order()))[i+1:]
    for j in a:
        n += 1
        if nx.has_path(g,i,j):
            path += nx.shortest_path_length(g,i,j)
ave_path_length = path/n
print('average path length :',ave_path_length)

#3.(b-5)---------------------------------
cc = len(list(max(nx.connected_components(g))))
print('nG :',cc)

#3.(c)---------------------------------
emax = (75*74)/2
print('e/emax :',g.size()/emax)

NameError: name 'pd' is not defined